In [ ]:
import subprocess, sys, os

dpsim_root_dir = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
sys.path.insert(0, os.path.join(dpsim_root_dir, 'python/src/dpsim/'))
sys.path.insert(0, os.path.join(dpsim_root_dir, 'build'))

from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
from villas.dataprocessing.timeseries import TimeSeries as ts
import villas.dataprocessing.plottools as pt
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

sim_name_fault_emt = "EMT_Kundur2Areas_Fault"
sim_name_s1 = "Cosim_Kundur2Areas_Fault_S1_zoh_"
sim_name_s2 = "Cosim_Kundur2Areas_Fault_S2_zoh_"

H_v = [0.0002, 0.0003]
prefixes = ['0.0002', '0.0003']
prefix_labels = {'0.0002': '2e-4', '0.0003': '3e-4'}

dpsim_result_file_fault = 'logs/' + sim_name_fault_emt + '/' + sim_name_fault_emt + '.csv'
ts_dpsim_fault_emt = read_timeseries_csv(dpsim_result_file_fault)

timestep_common = 10e-5
varname_dpsim = 'N8.V'
nominal_voltage = 230000

t_begin = 0.0
t_end = 2
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)
time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

#plot parameters
width = 6
height = 4

fig, axes = plt.subplots(figsize=(width, height))

#convert dpsim voltage to magnitude value and per-unit for comparison with psat
dpsim_emt_values = ts_dpsim_fault_emt[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)

#second axis of first plot
roi_start_second=0.095
roi_end_second=0.105
ylim = axes.get_ylim()
roi = range(int(roi_start_second/timestep_common), int(roi_end_second/timestep_common))
ax2 = axes.inset_axes([0.35, 0.45, .55, .55])
# x1, x2, y1, y2 = roi_start_second, roi_end_second, -0.4, 1
x1, x2, y1, y2 = roi_start_second, roi_end_second, -0.5, 1
mark_inset(axes, ax2, loc1=2, loc2=3, fc="none", color='black')

for pref in prefixes:
    dpsim_result_file_fault_s1 = 'logs/' + sim_name_s1 + pref + '/' + sim_name_s1 + pref + '.csv'
    ts_dpsim_fault_s1_emt = read_timeseries_csv(dpsim_result_file_fault_s1)
                                 
    dpsim_result_file_fault_s2 = 'logs/' + sim_name_s2 + pref + '/' + sim_name_s2 + pref + '.csv'
    ts_dpsim_fault_s2_emt = read_timeseries_csv(dpsim_result_file_fault_s2)

     #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    dpsim_emt_values_s1 = ts_dpsim_fault_s1_emt[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)
    dpsim_emt_values_s2 = ts_dpsim_fault_s2_emt[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)

    # plt.subplot(1, 2, 1)
    axes.plot(time, dpsim_emt_values_s1, '--', label='EMT - Cosimulation ' + prefix_labels[pref])
    # plt.plot(time, dpsim_emt_values_s2, '--', label='EMT - DPsim - S2')
    axes.set_ylim((-1.5,3))
    axes.set_xlim([0.05, 0.25])
    axes.set_xlabel("time (s)")
    axes.set_ylabel(varname_dpsim + " (pu)")

    ax2.plot(time[roi], dpsim_emt_values_s1[roi], '--')
    ax2.set_xlim(x1, x2)
    ax2.set_ylim(y1, y2)

plt.tight_layout()

axes.plot(time, dpsim_emt_values, label='EMT - Monolithic')
ax2.plot(time[roi], dpsim_emt_values[roi])
# axes.grid()
plt.legend(loc='lower right')
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax2.tick_params(top=False, bottom=False, left=False, right=False, labelleft=True, labelbottom=True)
ax2.grid()
plt.savefig('Kundur2Areas_dyn_EMT_cosim_attributes_fault_{}_comparison.pdf'.format(varname_dpsim))

In [ ]:
from villas.dataprocessing.timeseries import TimeSeries as ts
import villas.dataprocessing.plottools as pt

sim_name_fault_emt = "EMT_Kundur2Areas_Fault"
sim_name_s1 = "Cosim_Kundur2Areas_Fault_S1_zoh_"
sim_name_s2 = "Cosim_Kundur2Areas_Fault_S2_zoh_"

H_v = [0.0002, 0.0003]
prefixes = ['0.0002', '0.0003']

dpsim_result_file_fault = 'logs/' + sim_name_fault_emt + '/' + sim_name_fault_emt + '.csv'
ts_dpsim_fault_emt = read_timeseries_csv(dpsim_result_file_fault)

timestep_common = min(H_v)
varname_dpsim = 'N8.V'
nominal_voltage = 230000

t_begin = 0.0
t_end = 2
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)
time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

#plot parameters
width = 6
height = 4

plt.figure(figsize=(width, height))

#convert dpsim voltage to magnitude value and per-unit for comparison with psat
dpsim_emt_values = ts_dpsim_fault_emt [varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)
plt.plot(time, dpsim_emt_values, label='EMT - Monolithic')

for pref in prefixes:
    dpsim_result_file_fault_s1 = 'logs/' + sim_name_s1 + pref + '/' + sim_name_s1 + pref + '.csv'
    ts_dpsim_fault_s1_emt = read_timeseries_csv(dpsim_result_file_fault_s1)
                                 
    dpsim_result_file_fault_s2 = 'logs/' + sim_name_s2 + pref + '/' + sim_name_s2 + pref + '.csv'
    ts_dpsim_fault_s2_emt = read_timeseries_csv(dpsim_result_file_fault_s2)

     #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    dpsim_emt_values_s1 = ts_dpsim_fault_s1_emt[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)
    dpsim_emt_values_s2 = ts_dpsim_fault_s2_emt[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)

    # plt.subplot(1, 2, 1)
    plt.plot(time, dpsim_emt_values_s1, '--', label='EMT - Cosimulation')
    # plt.plot(time, dpsim_emt_values_s2, '--', label='EMT - DPsim - S2')
    plt.legend(loc='lower right')
    # plt.ylim((-1.5,2.5))
    # plt.xlim([0.05, 0.25])
    plt.ylim((-0.5,1))
    plt.xlim([0.095, 0.105])
    plt.xlabel("time (s)")
    plt.ylabel(varname_dpsim + " (pu)")
        
plt.grid()
plt.show()

In [ ]:

sim_name_fault_dp = "DP_Kundur2Areas_SS"
sim_name_s1_dp = "DP_Cosim_Kundur2Areas_SS_S1_zoh_"
sim_name_s2_dp = "DP_Cosim_Kundur2Areas_SS_S2_zoh_"

H_v = [0.0001, 0.0002, 0.0003]
prefixes = ['0.0001', '0.0002', '0.0003']
prefix_labels = {'0.0001': '1e-4', '0.0002': '2e-4', '0.0003': '3e-4'}

dpsim_result_file_fault = 'logs/' + sim_name_fault_dp + '/' + sim_name_fault_dp + '.csv'
ts_dpsim_fault_dp = read_timeseries_csv(dpsim_result_file_fault)

timestep_common = min(H_v)
varname_dpsim = 'N8.V'
nominal_voltage = 230000

t_begin = 0.0
t_end = 2
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)
time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

#plot parameters
width = 6
height = 4

plt.figure(figsize=(width, height))

#convert dpsim voltage to magnitude value and per-unit for comparison with psat
dpsim_dp_values = np.abs(ts_dpsim_fault_dp[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx]) / nominal_voltage

for pref in prefixes:
    dpsim_result_file_fault_s1 = 'logs/' + sim_name_s1_dp + pref + '/' + sim_name_s1_dp + pref + '.csv'
    ts_dpsim_fault_s1_dp = read_timeseries_csv(dpsim_result_file_fault_s1)
                                 
    dpsim_result_file_fault_s2 = 'logs/' + sim_name_s2_dp + pref + '/' + sim_name_s2_dp + pref + '.csv'
    ts_dpsim_fault_s2_dp = read_timeseries_csv(dpsim_result_file_fault_s2)

    #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    dpsim_dp_values_s1 = np.abs(ts_dpsim_fault_s1_dp[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx]) / nominal_voltage
    dpsim_dp_values_s2 = np.abs(ts_dpsim_fault_s2_dp[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx]) / nominal_voltage

    # plt.subplot(1, 2, 1)
    plt.plot(time, dpsim_dp_values_s1, '--', label='DP - Cosimulation ' + prefix_labels[pref])
    # plt.plot(time, dpsim_emt_values_s2, '--', label='EMT - DPsim - S2')
    # plt.ylim((-1.5,2.5))
    # plt.xlim([0.05, 0.25])
    # plt.ylim((-0.5,1))
    # plt.xlim([0.095, 0.105])
    plt.xlabel("time (s)")
    plt.ylabel(varname_dpsim + " (pu)")
        
plt.plot(time, dpsim_dp_values, label='DP - Monolithic')
plt.grid()
plt.legend(loc='lower right')
plt.show()